In [2]:
%env JAVA_HOME=/Library/Java/JavaVirtualMachines/jdk1.8.0_112.jdk/Contents/Home

env: JAVA_HOME=/Library/Java/JavaVirtualMachines/jdk1.8.0_112.jdk/Contents/Home


Let's create a clause and see what is inside

In [3]:
from logic import *
from dataset import InterpretationDataset

In [4]:
clause = Clause.parse("!human(socrates),!sibling(socrates,Y),!human(sonOf(Y)),mortal(Y)")

print("clause {}".format(clause))

for literal in clause:
    print("{}ground literal {} contains following arguments".format("" if literal.isGround() else "non-", literal))
    for term, idx in zip(literal, range(0, len(literal))):
        print("\t{}-th argument:\t{}".format(idx, term))

clause !human(socrates) | !human(sonOf(Y)) | !sibling(socrates, Y) | mortal(Y).
ground literal !human(socrates) contains following arguments
	0-th argument:	socrates
non-ground literal !sibling(socrates, Y) contains following arguments
	0-th argument:	socrates
	1-th argument:	Y
non-ground literal !human(sonOf(Y)) contains following arguments
	0-th argument:	sonOf(Y)
non-ground literal mortal(Y) contains following arguments
	0-th argument:	Y


Let's see how to go throught a datset of interpretations.

In [12]:
data = InterpretationDataset(os.path.sep.join(['.','data','lectureGraphs']))

In [13]:
for sample in data:
    print(sample)

- : {edge(11, 13), edge(14, 11), edge(14, 13), edge(12, 13), edge(12, 14)}
- : {edge(21, 22), edge(22, 23), edge(23, 21), edge(22, 24)}
- : {edge(33, 31), edge(31, 32), edge(32, 33), edge(32, 34), edge(33, 34)}
+ : {edge(43, 41), edge(41, 42), edge(42, 44), edge(43, 44)}
+ : {edge(51, 52), edge(52, 53)}
+ : {edge(63, 67), edge(67, 66), edge(66, 61), edge(61, 62), edge(62, 63), edge(63, 64), edge(64, 65), edge(65, 68), edge(68, 67)}
- : {edge(11, 13), edge(14, 11), edge(14, 13), edge(12, 13), edge(12, 14)}


Each sample, i.e. observation, consists of a class, e.g. pos/neg, and a set of atoms which holds. Their are easily accesible.

In [14]:
sample = tuple(data)[0]
print('class of this sample is\t{}'.format("pos" if sample.positiveClass else "neg"))
print('the sample contains following facts')
for atom in sample:
    print("\t{}".format(atom))

class of this sample is	neg
the sample contains following facts
	edge(11, 13)
	edge(14, 11)
	edge(14, 13)
	edge(12, 13)
	edge(12, 14)


Keep in mind that two expressions even parsed by the same process are not the same instance, thus 'is' comparator does not return what is probably expected from the first sight. Therefore, it is advised to use "==" comparator. See following examples and compare the output to what you expect.

In [19]:
c1 = Clause.parse("p(X,a)")
c2 = Clause.parse("p(X,a)")

print(c1.literals[0] is c2.literals[0])

print(c1 is c2)
print(c1 == c2)

for t1, t2 in zip(c1.literals[0].atom.terms, c2.literals[0].atom.terms):
    print("{}\tvs\t{}".format(t1, t2))
    print("\t{}".format(t1 is t2))
    print("\t{}".format(t1 == t2))

False
False
True
X	vs	X
	False
	True
a	vs	a
	False
	True


Despite the fact that two clauses are logically equal, they are not equal by the "==" operator. The operator is implemented by lexical comparison only.

In [16]:
c3 = Clause.parse("p(X,Y)")
c4 = Clause.parse("p(Z,W)")
print(c3 == c4)

False


The easiest way to apply a substitution to a clause is to create a substitution, i.e. a dictionary of terms indexed by variables. 

In [17]:
c5 = Clause.parse("siblings(X,Y,Z,W)")
x = Variable("X")
y = Variable("Y")
z = Variable("Z")
issac = Constant("issac")
kain = Constant("kain")
substitution = {x : issac, y : kain, z : x}
substitutedClause = c5.substitute(substitution)
print(substitutedClause)

siblings(issac, kain, X, W).


Note here one important thing. The substituted clause should look, by the definition of substitution in FOL, as sibling(issac, kain, issac, W). However, this library supports, and also expects, flat substitutions only. Therefore, no chaning is applied during the substitution process, e.g. the result is siblings(issac,kain,X,W) since no chaning {Z -> X, X -> issac} is done. If you want such behavior, then flatten the substitution first, e.g. {Z -> issac, X -> issac}. However, in our tutorials, we do not need such functionality.